**Inżynieria cech - missing values**

In [16]:
# import potrzebnych bibliotek
import numpy as np
import pandas as pd

In [4]:
# wczytanie danych titanic
df = pd.read_csv("titanic.txt", sep=",")

In [5]:
# sprawdzenie 20 pierwszych obserwacji
print(df.head(20))

    pclass  survived                                               name  \
0        1         1                      Allen, Miss. Elisabeth Walton   
1        1         1                     Allison, Master. Hudson Trevor   
2        1         0                       Allison, Miss. Helen Loraine   
3        1         0               Allison, Mr. Hudson Joshua Creighton   
4        1         0    Allison, Mrs. Hudson J C (Bessie Waldo Daniels)   
5        1         1                                Anderson, Mr. Harry   
6        1         1                  Andrews, Miss. Kornelia Theodosia   
7        1         0                             Andrews, Mr. Thomas Jr   
8        1         1      Appleton, Mrs. Edward Dale (Charlotte Lamson)   
9        1         0                            Artagaveytia, Mr. Ramon   
10       1         0                             Astor, Col. John Jacob   
11       1         1  Astor, Mrs. John Jacob (Madeleine Talmadge Force)   
12       1         1     

In [6]:
# wstępna inspekcja 20 wierszy
df.head(20).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pclass     20 non-null     int64 
 1   survived   20 non-null     int64 
 2   name       20 non-null     object
 3   sex        20 non-null     object
 4   age        20 non-null     object
 5   sibsp      20 non-null     int64 
 6   parch      20 non-null     int64 
 7   ticket     20 non-null     object
 8   fare       20 non-null     object
 9   cabin      20 non-null     object
 10  embarked   20 non-null     object
 11  boat       20 non-null     object
 12  body       20 non-null     object
 13  home.dest  20 non-null     object
dtypes: int64(4), object(10)
memory usage: 2.3+ KB


In [7]:
# sprawdzenie liczby cech
print("Liczba cech:", df.shape[1])

Liczba cech: 14


In [8]:
# w tym zbiorze danych braki oznaczone są ?
# zamiana ? na nan
df = df.replace("?", np.nan)

In [9]:
# Liczba braków w każdej kolumnie
df.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64



Najwięcej braków danych jest w kolumnie body. Jest to kolumna, w której znajduje się identyfikator odnalezionego ciała. Z powodu tego, że była zbyt mała liczba łodzi ratunkowych (informacja z filmiku), większość ciał mogła odpłynąć i już nigdy nie zostać odnaleziona.

Bardzo dużo braków danych jest także w kolumnie 'boat', co potwierdza informacje z filmiku o braku odpowiedniej liczby łodzi ratunkowych. Ten typ braku danych to MNAR (Missing Not At Random), ponieważ brak liczby łodzi jest związany z danymi historycznymi.

Sytuacja ma się podobnie z kolumną 'cabin', w której znajduje się 1014 braków obserwacji. Może być to związane po prostu z tym, że jak ktoś rezerwował nizszą klasę kabiny to nie było to odnotowywane, co wiąże sie bezpośrednio z tym, że te dane będą należeć do kategorii MNAR.

Natomiast kolumna 'home.dest' najprawdopodobniej jest MAR, ponieważ brak jej wartości można wyjaśnić innymi zmiennymi, np. klasą biletu i miejscem pochodzenia pasażera.


In [10]:
# procent braków w każdej kolumnie
round(df.isnull().mean() * 100, 2)

pclass        0.00
survived      0.00
name          0.00
sex           0.00
age          20.09
sibsp         0.00
parch         0.00
ticket        0.00
fare          0.08
cabin        77.46
embarked      0.15
boat         62.87
body         90.76
home.dest    43.09
dtype: float64

Kolumna body będzie wymagała specjalnego traktowania. Będziemy oznaczać ją binarnie przez bardzo duże braki danych.

In [11]:
df['KolumnaNull'] = np.where(df['body'].isnull(), 1, 0)
df.groupby(['survived'])['KolumnaNull'].mean()

survived
0    0.850433
1    1.000000
Name: KolumnaNull, dtype: float64



Jeżeli człowiek nie przeżył to w 85% jego ciała i tak nie odnaleziono. Natomiast jeżeli człowiek przeżył to w 100% jego ciała nie odnaleziono, ponieważ uciekł z miejsca tragedii i nie trzeba było szukać jego zwłok.

Dane w kolumnie 'body' są MNAR (Missing Not At Random), ponieważ zależą od tego czy ktoś przeżył.


In [13]:
df.groupby('survived')['boat'].apply(lambda x: x.isnull().mean())

survived
0    0.988875
1    0.046000
Name: boat, dtype: float64



Jeżeli człowiek nie przeżyl to w prawie 99% nie posiadał łodzi ratunkowej. Natomiast jeżeli człowiek przeżył to tylko w 4.6% nie posiadał łodzi ratunkowej. Istnieje więc bardzo silna zależność między posiadaniem łodzi a przeżyciem.


Jeżeli człowiek nie przeżył to w 23% nie ustalono jego wieku, a jedynie 14.6% osób, które przeżyły zostały zidentyfikwoane pod względem wieku. Dużo osób mogło zginąć ze swoimi dokumentami osobistymi, co pozwoliło ich zidentyfikować.

In [15]:
df['age'] = df['age'].astype(float)
df['fare'] = df['fare'].astype(float)

df['age'] = df['age'].fillna(df['age'].median())
df['fare'] = df['fare'].fillna(df['fare'].mean())

df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)


C:\Users\jakub\AppData\Local\Temp\ipykernel_28348\2945103142.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)


'age' oraz 'fare' można uzupełnić za pomocą średniej, bądź mediany, natomiast 'embarked' przez to, że jest zmienną kategoryczną można zastąpić najczęstszym wynikiem w danych.